In [36]:
import cv2
import cv_exp.basic
import cv_exp.basic as cv
import cv_exp.pupil_detection as pupil_detection
import cv_exp.draw as draw
import cv_exp.log as log
import cv_exp.take as take
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os
from PIL import Image
import pandas as pd
from scipy.cluster.vq import kmeans, vq
import matplotlib.pyplot as plt
import plotly.express as px


model_face_detection = '/home/roopesh/Desktop/MediaPipe_new_cv_exp/cv-exp-framework-master/data/models/face_detection_front.tflite'
model_face_landmarks ='/home/roopesh/Desktop/MediaPipe_new_cv_exp/cv-exp-framework-master/data/models/face_landmark.tflite'
model_iris_landmarks = '/home/roopesh/Desktop/MediaPipe_new_cv_exp/cv-exp-framework-master/data/models/iris_landmark.tflite'

iris_detector = pupil_detection.IrisDetectorMP(model_face_detection_path=model_face_detection,
                                                   model_face_landmarks_path=model_face_landmarks,
                                                   model_iris_landmarks_path=model_iris_landmarks)

# def Mediapipe_Iris_Color(path):
#     '''

#     This function takes the image in path and Using Mediapipe calculates the Annular Mask of the images,
#     then calculates the HSV color of Annular mask.
#     and returns the dominant Iris colour percentage in a data frame.
#     '''



def rgb_to_hsv(r, g, b):
    '''
    Convert RGB to HSV
    returns h,s,v
    '''

    r, g, b = r/255.0, g/255.0, b/255.0
    mx = max(r, g, b)
    mn = min(r, g, b)
    df = mx-mn
    if mx == mn:
        h = 0
    elif mx == r:
        h = (60 * ((g-b)/df) + 360) % 360
    elif mx == g:
        h = (60 * ((b-r)/df) + 120) % 360
    elif mx == b:
        h = (60 * ((r-g)/df) + 240) % 360
    if mx == 0:
        s = 0
    else:
        s = (df/mx)*100
    v = mx*100
    return h, s, v




path='/home/roopesh/Desktop/New_Corpus/test/front (4).jpg'

image = cv2.imread(path)
image_bgr_rgb = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # converting from BGR to RGB
out_ = image_bgr_rgb.copy()
out_pupile_ = image_bgr_rgb.copy()
result = iris_detector.detect(image_bgr_rgb)

if not result is None:

    detected_pupils = result.detected_pupils,
    face_rect_detection = result.face_rect_detection_from_landmarks,
    face_rect_landmarks = result.face_rect_from_landmarks,
    face_landmarks = result.face_landmarks,
    face_rect_detection_from = result.face_rect_from_detection,
    face_landmarks_from_detection = result.face_landmarks_from_detection,
    left_iris_landmarks = result.left_iris_landmarks,
    left_eyelid_landmarks = result.left_eyelid_landmarks,
    right_iris_landmarks = result.right_iris_landmarks,
    right_eyelid_landmarks = result.right_eyelid_landmarks,
    left_iris_segmentation = result.left_iris_segmentation,
    right_iris_segmentation = result.right_iris_segmentation,
    eyes_landmarks = result.eyes_landmarks

    # drawing (Optional)

    out_pupile = draw.draw_pupils(out_pupile_, result.detected_pupils.left, result.detected_pupils.right, color_center = (255, 255, 255), color_contour = (255, 255, 255))
    out_pupile = draw.draw_rect(out_pupile_, result.face_rect_detection_from_landmarks, thickness=6, color=(255, 0, 0))
    # out_pupile = draw.draw_list_pointd(out_pupile_, result.left_iris_landmarks,radius=6,color = (255, 255, 255))
    # out_pupile= draw.draw_contour(out_pupile_, result.left_iris_segmentation,  thickness=1,color=(0, 0, 255))
    # out_pupile = draw.draw_list_pointd(out_pupile_, result.left_eyelid_landmarks,radius=6,color = (255, 0, 0))

    # Extraction of specific Points From Left Iris Landmarks (0,1) : Left Iris from Coder's Point of view

    left_iris_tuple_of_tuple_Tolist = ([PointD_elements for PointD in left_iris_landmarks for PointD_elements in PointD])
    left_iris_points = [(pointD.x, pointD.y) for pointD in left_iris_tuple_of_tuple_Tolist]

    center_point_LeftIris=(np.round(left_iris_points[0]))
    for left_center_x,left_center_y in [center_point_LeftIris]:
         lx, ly = left_center_x,left_center_y

    left_point_LeftIris=(np.round(left_iris_points[1]))
    for leftIris_left_x,LeftIris_left_y in [left_point_LeftIris]:
        LIL_x, LIL_y = leftIris_left_x, LeftIris_left_y

    right_point_LeftIris=(np.round(left_iris_points[3]))
    for rightIris_x,rightIris_y in [right_point_LeftIris]:
        RIL_x, RIL_y = rightIris_x, rightIris_y

    Top_point_LeftIris=(np.round(left_iris_points[2]))
    for TopIris_x,TopIris_y in [Top_point_LeftIris]:
        TI_x, TI_y = TopIris_x, TopIris_y

    # Bottom_point_LeftIris=(np.round(left_iris_points[4]))
    # for BottomIris_x,BottomIris_y in [Bottom_point_LeftIris]:
    #     BI_x, BI_y = BottomIris_x, BottomIris_y


    # Distance Formula to calculate the distance between center to left_Iris_ points to get radius of iris
    distance_left_iris_center = np.sqrt((lx - LIL_x) ** 2 + (ly - LIL_y) ** 2)

    # First Circular mask for entire iris

    mask = np.zeros(out_pupile_.shape, dtype=np.uint8)      # Result from Mediapipe added here.
    center = (int(lx), int(ly))
    radius = (int((distance_left_iris_center) * 0.80))
    left_mask = cv2.circle(mask, center, radius,[255,255,255], -1 )
    result = cv2.bitwise_and(out_pupile_, left_mask)
    result[mask == 0] = 255                                 # adding background Color- white

    # cropping iris area from the first circular masked image

    x = int((RIL_x)+3)               # x-axis --Landmarks points from top- right point from left iris, +3 for adjusting the size
    y = int((TI_y)+3)                # y-axis --Landmarks points from top_ left of left iris, +3 for adjusting the size
    h = int(radius*2.35)-3               # height --Radius of iris times 2.35 for adjustment,-3 for adjustment
    w = int(radius*2.35)             # width  --Radius of iris times 2.35 for adjustment, -1 for adjustment
    crop = result[y:y+h, x:x+w]

    # second Circular mask to take inner pupil area alone

    mask1 = np.zeros(out_pupile_.shape, dtype=np.uint8)         # Result from Mediapipe added here for second mask.
    center1 = (int(lx), int(ly))
    radius1 = (int((distance_left_iris_center)* 0.50))          # to scrap pupil area and other reflection area
    left_mask1 = cv2.circle(mask1, center1, radius1, (255, 255, 255), -1 )
    result1 = cv2.bitwise_and(out_pupile_, left_mask1-255)      # subtracting white region over masked area
    result1[mask1 == 0] = 255                                   # adding background Color- white

    # Cropping the inner pupil area from the second circular masked image (size should be same as the first masked and cropped result )

    x = int(RIL_x)+3         # x-axis --Landmarks points from top- right point from left iris, +3 for adjusting the size
    y = int(TI_y)+3          # y-axis --Landmarks points from top_ left of left iris, +3 for adjusting the size
    h = int(radius*2.35)-3   # height --Radius of iris times 2.35,-3 for adjustment
    w = int(radius*2.35)     # width  --Radius of iris times 2.35, -1 for adjustment
    crop1 = result1[y:y+h, x:x+w]

    # Bitwise and operation to omit the inner pupil alone and take the rest of the iris area

    adding_both_cropped_image = cv2.bitwise_and(crop, crop1)

    # Removing NOISE from the Adding_both_cropped_image (<=10,  >=240) in all channel

    final_result = np.array(adding_both_cropped_image)
    mask = np.all(final_result <= [10, 10, 10], axis=2)
    final_result[mask] = [0, 0, 0]
    mask = np.all(final_result >= [240, 240, 240], axis=2)
    final_result[mask] = [0, 0, 0]
    annular_result = final_result                   # the final result of the iris area from mediapipe

    # Additional Cropping the iris masked from the Annular_result to make 3 sub cropped images

    x, y = final_result.shape[:2]
    half = x//2
    top_third_= x//3
    Upper_Third = final_result[:top_third_]               # Upper third of the image
    top_3 = final_result[top_third_:half]
    bottom = final_result[half:]
    x1, y1 = bottom.shape[:2]
    bottom_third = x1//3
    bo_3 = bottom[:bottom_third]
    half1 = x1//3
    Lower_Third = bottom[half1:]                          # Lower third of the image
    Middle_Third = np.vstack((top_3, bo_3))               # middle third of the image

    dict01 = dict()
    dict01['Upper_Third'] = Upper_Third
    dict01['Middle_Third'] = Middle_Third
    dict01['Lower_Third'] = Lower_Third

    # Printing the cropped iris area from the Annular_result (optional)

    annular_result1 = cv2.cvtColor(annular_result, cv2.COLOR_RGB2BGR)
    Upper_Third1 = cv2.cvtColor(Upper_Third, cv2.COLOR_RGB2BGR)
    Middle_Third1 = cv2.cvtColor(Middle_Third, cv2.COLOR_RGB2BGR)
    Lower_Third1 = cv2.cvtColor(Lower_Third, cv2.COLOR_RGB2BGR)

    out_pupile_1 = cv2.cvtColor(out_pupile, cv2.COLOR_RGB2BGR)
    out_pupile_1 = cv2.putText(out_pupile_1, "Left Iris", (int(TI_x), int(TI_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

#     cv2.imwrite(f'/home/roopesh/Desktop/new_cvexp_results/V2.0/folder/annular_Iris_/{file}',annular_result1)
#     cv2.imwrite(f'/home/roopesh/Desktop/new_cvexp_results/V2.0/folder/Iris_original_/{file}',out_pupile_1)
#     cv2.imwrite(f'/home/roopesh/Desktop/new_cvexp_results/V2.0/folder/Upper_Third/{file}',Upper_Third1)
#     cv2.imwrite(f'/home/roopesh/Desktop/new_cvexp_results/V2.0/folder/Middle_Third/{file}',Middle_Third1)
#     cv2.imwrite(f'/home/roopesh/Desktop/new_cvexp_results/V2.0/folder/Lower_Third/{file}',Lower_Third1)

    
    #   Reading Image in PIL format for dominant color detection.

    dominantColour = []

    for keys, values in dict01.items():
        pil_image = Image.fromarray(values)
        imarray = np.array(pil_image.getdata())
        df = pd.DataFrame(imarray, columns=['RED', 'GREEN', 'BLUE'])
        (df[(df['RED'] >= 50) | (df['GREEN'] >= 50) | (df['BLUE'] >= 50)])  # For value greater than 50

         # K-Means Clustering using Scipy

        cluster_size = 3
        cluster_centers, _ = kmeans(df.values.astype(float), cluster_size)
        index, _ = vq(df.values.astype(float), cluster_centers)
        count = np.bincount(index)
        sum = np.sum(count)
        i, Blue, Green, Brown, Other = 0, 0, 0, 0, 0
        for row in cluster_centers:
            r = row[0]
            g = row[1]
            b = row[2]
            h, s, v = rgb_to_hsv(r, g, b)   # calling the function to convert RGB to HSV
            percent = count[i]*100/sum
            if h >= 170 and h < 270:
                Blue += percent
            elif h >= 60 and h < 170:
                Green += percent
            elif h >= 300 or h < 40:
                Brown += percent
            else:
                Other += percent
            i+=1

        # Results of Dominant Color Detection in dataframe

        data = [[path, str(keys), int(Blue), int(Green), int(Brown), int(Other) ]]
        dominantColour_ = pd.DataFrame(data, columns=['File_name', 'Image_Position', 'Blue', 'Green', 'Brown','Other'])
        dominantColour.append(dominantColour_)


    print(dominantColour[0])

else: print("No face detected :", file)

 # **************************  END of Dominant Colour Detection **************************


def rgb_to_hsv(r, g, b):
    '''
    Convert RGB to HSV
    returns h,s,v
    '''

    r, g, b = r/255.0, g/255.0, b/255.0
    mx = max(r, g, b)
    mn = min(r, g, b)
    df = mx-mn
    if mx == mn:
        h = 0
    elif mx == r:
        h = (60 * ((g-b)/df) + 360) % 360
    elif mx == g:
        h = (60 * ((b-r)/df) + 120) % 360
    elif mx == b:
        h = (60 * ((r-g)/df) + 240) % 360
    if mx == 0:
        s = 0
    else:
        s = (df/mx)*100
    v = mx*100
    return h, s, v

# if __name__ == '__main__':

# # ----------Load Image Directory -------------
# # images_files_dir= '/home/roopesh/Desktop/new_cvexp_results/V2.0/New_Green_Eyes/Green_Iris_Training_Image/' # Directory of the images

# # images_files_dir= '/home/roopesh/Desktop/new_cvexp_results/V2.0/New_Brown_Eyes/Brown_Iris_Training_Image_V4.0/'
# # images_files_dir= '/home/roopesh/Desktop/new_cvexp_results/V2.0/New_Blue_Eyes/Blue_Iris_Training_Image (copy)/'
# # images_files_dir= '/home/roopesh/Desktop/new_cvexp_results/V2.0/New_Green_Eyes/Green_Iris_Training_Image_V4.0/'

# # images_files_dir= '/home/roopesh/Desktop/New_Corpus_v3/Brown_V4.0/'
# # images_files_dir= '/home/roopesh/Desktop/New_Corpus_v3/Blue/'
# images_files_dir= '/home/roopesh/Desktop/New_Corpus_v3/Green/'

# files = os.listdir(images_files_dir)
# # sorted_file=sorted(files, key=lambda x : int(x[5:-4]))
# for idx, file in enumerate(files):
#     image_path = images_files_dir + file
#     # print('debug', file)
#     mediapipe_iris_color_chart = Mediapipe_Iris_Color(image_path)           # Calling the Mediapipe_Iris_Color(image_path)
#     print(mediapipe_iris_color_chart)





                                           File_name Image_Position  Blue  \
0  /home/roopesh/Desktop/New_Corpus/test/front (4...    Upper_Third     0   

   Green  Brown  Other  
0      0    100      0  


In [29]:
pd.DataFrame(df[(df['RED'] >= 20) & (df['GREEN'] >= 20) & (df['BLUE'] >= 20)])is True


False

In [3]:
print(type(dominantColour))

<class 'list'>


In [4]:
cat= pd.concat(dominantColour)
cat

,File_name,Image_Position,Blue,Green,Brown,Other
0,/home/roopesh/Desktop/New_Corpus/Brown/front (...,Upper_Third,100,0,0,0
0,/home/roopesh/Desktop/New_Corpus/Brown/front (...,Middle_Third,0,0,70,29
0,/home/roopesh/Desktop/New_Corpus/Brown/front (...,Lower_Third,0,0,42,57


In [5]:
# cat=cat.reset_index(drop=True)
cat=cat.set_index('Image_Position')


In [6]:
cat

,File_name,Blue,Green,Brown,Other
Image_Position,,,,,
Upper_Third,/home/roopesh/Desktop/New_Corpus/Brown/front (...,100,0,0,0
Middle_Third,/home/roopesh/Desktop/New_Corpus/Brown/front (...,0,0,70,29
Lower_Third,/home/roopesh/Desktop/New_Corpus/Brown/front (...,0,0,42,57


In [93]:
dict1=cat.to_dict()

In [16]:
dict1={'File_name': {'Upper_Third': '/home/roopesh/Desktop/New_Corpus/Brown/front (3).jpg',
  'Middle_Third': '/home/roopesh/Desktop/New_Corpus/Brown/front (3).jpg',
  'Lower_Third': '/home/roopesh/Desktop/New_Corpus/Brown/front (3).jpg'},
 'Blue': {'Upper_Third': 0, 'Middle_Third': 0, 'Lower_Third': 0},
 'Green': {'Upper_Third': 0, 'Middle_Third': 0, 'Lower_Third': 0},
 'Brown': {'Upper_Third': 100, 'Middle_Third': 24, 'Lower_Third': 100},
 'Other': {'Upper_Third': 0, 'Middle_Third': 75, 'Lower_Third': 0}} 

In [17]:
def check_5Percent_range(x,y):
    flg=False
    if x < y:
        if x+(x*0.05) >= y:
            flg=True
    elif x > y:
        if x - (x*0.05) <= y:
            flg=True
    elif x==y:
        flg=True
    return flg

# Upper Lower Middle Third  

Blue_All_Equal=check_5Percent_range(dict1['Blue']['Upper_Third'],dict1['Blue']['Middle_Third']) and check_5Percent_range(dict1['Blue']['Upper_Third'],dict1['Blue']['Lower_Third'])and check_5Percent_range(dict1['Blue']['Lower_Third'],dict1['Blue']['Middle_Third'])
Brown_All_Equal=check_5Percent_range(dict1['Brown']['Upper_Third'],dict1['Brown']['Middle_Third']) and check_5Percent_range(dict1['Brown']['Upper_Third'],dict1['Brown']['Lower_Third'])and check_5Percent_range(dict1['Brown']['Lower_Third'],dict1['Brown']['Middle_Third'])
Green_All_Equal=check_5Percent_range(dict1['Green']['Upper_Third'],dict1['Green']['Middle_Third']) and check_5Percent_range(dict1['Green']['Upper_Third'],dict1['Green']['Lower_Third'])and check_5Percent_range(dict1['Green']['Lower_Third'],dict1['Green']['Middle_Third'])
Other_All_Equal = check_5Percent_range(dict1['Other']['Upper_Third'],dict1['Other']['Middle_Third']) and check_5Percent_range(dict1['Other']['Upper_Third'],dict1['Other']['Lower_Third'])and check_5Percent_range(dict1['Other']['Lower_Third'],dict1['Other']['Middle_Third'])

# Upper and Middle Third

Blue_Upper_Middle_Equal = check_5Percent_range(dict1['Blue']['Upper_Third'],dict1['Blue']['Middle_Third'])
Green_Upper_Middle_Equal = check_5Percent_range(dict1['Green']['Upper_Third'],dict1['Green']['Middle_Third'])
Brown_Upper_Middle_Equal = check_5Percent_range(dict1['Brown']['Upper_Third'],dict1['Brown']['Middle_Third'])
Other_Upper_Middle_Equal = check_5Percent_range(dict1['Other']['Upper_Third'],dict1['Other']['Middle_Third'])

Upper_Middle = Blue_Upper_Middle_Equal is False or True and Green_Upper_Middle_Equal is False or True and Brown_Upper_Middle_Equal is False or True and Other_Upper_Middle_Equal is False or True


# Middle and Lower Third

Blue_Middle_Lower_Equal = check_5Percent_range(dict1['Blue']['Lower_Third'],dict1['Blue']['Middle_Third'])
Green_Middle_Lower_Equal =check_5Percent_range(dict1['Green']['Lower_Third'],dict1['Green']['Middle_Third'])
Brown_Middle_Lower_Equal = check_5Percent_range(dict1['Brown']['Lower_Third'],dict1['Brown']['Middle_Third'])
Other_Middle_Lower_Equal =check_5Percent_range(dict1['Other']['Lower_Third'],dict1['Other']['Middle_Third'])

Middle_Lower = (Blue_Middle_Lower_Equal and Green_Middle_Lower_Equal and Brown_Middle_Lower_Equal and
                Other_Middle_Lower_Equal)

#  Upper and Lower Third

Blue_Upper_Lower_Equal = check_5Percent_range(dict1['Blue']['Upper_Third'],dict1['Blue']['Lower_Third'])
Green_Upper_Lower_Equal = check_5Percent_range(dict1['Green']['Upper_Third'],dict1['Green']['Lower_Third'])
Brown_Upper_Lower_Equal = check_5Percent_range(dict1['Brown']['Upper_Third'],dict1['Brown']['Lower_Third'])
Other_Upper_Lower_Equal = check_5Percent_range(dict1['Other']['Upper_Third'],dict1['Other']['Lower_Third'])

Upper_Lower = Blue_Upper_Lower_Equal is False and Green_Upper_Lower_Equal is False and Brown_Upper_Lower_Equal is False and Other_Upper_Lower_Equal is False

# Upper Middle Lower Not Equal  

all_conflict = Blue_All_Equal is False and Green_All_Equal is False and Brown_All_Equal is False and  Other_All_Equal is False

# 2.  Upper Middle Lower Agree

if Blue_All_Equal and Green_All_Equal and  Green_All_Equal and Other_All_Equal is True:    

    All_Equal_Blue=  (dict1['Blue']['Middle_Third'])
    All_Equal_green= (dict1['Green']['Middle_Third'])
    All_Equal_brown= (dict1['Brown']['Middle_Third'])
    All_Equal_other= (dict1['Other']['Middle_Third'])
   
    conflict_result= {'File_name': 'path','Blue': All_Equal_Blue,
                      'Green': All_Equal_green,'Brown': All_Equal_brown,'Other': All_Equal_other}
   
    print(conflict_result, 'No Conflicts')
    
    data1 = [[path, str('No Conflicts'), float(All_Equal_Blue), float(All_Equal_green), float(All_Equal_brown), float(All_Equal_Other) ]]
    conflict_result = pd.DataFrame(data1, columns=['File_name', 'Conflict_Type', 'Blue', 'Green', 'Brown','Other']).T
    print (conflict_result)
    
   
#  3. Upper/ Middle or Middle/Lower Agree

elif  Upper_Middle is False and Middle_Lower is True:
   
    upper_middle_blue =  (dict1['Blue']['Upper_Third']/2 + dict1['Blue']['Middle_Third']/2)
    upper_middle_green = (dict1['Green']['Upper_Third']/2 + dict1['Green']['Middle_Third']/2)
    upper_middle_brown = (dict1['Brown']['Upper_Third']/2 + dict1['Brown']['Middle_Third']/2)
    upper_middle_other = (dict1['Other']['Upper_Third']/2 + dict1['Other']['Middle_Third']/2)
   
    data1 = [[path, str('Conflict with Upper and Lower'), float(upper_middle_blue), float(upper_middle_green), float(upper_middle_brown), float(upper_middle_other) ]]
    conflict_result = pd.DataFrame(data1, columns=['File_name', 'Conflict_Type', 'Blue', 'Green', 'Brown','Other']).T
    print (conflict_result)
   
elif  Middle_Lower is False and Upper_Lower is True:
   
    Middle_Lower_blue = (dict1['Blue']['Middle_Third']/2 + dict1['Blue']['Lower_Third']/2)
    Middle_Lower_green = (dict1['Green']['Middle_Third']/2 + dict1['Green']['Lower_Third']/2)
    Middle_Lower_brown = (dict1['Brown']['Middle_Third']/2 + dict1['Brown']['Lower_Third']/2)
    Middle_Lower_other = (dict1['Other']['Middle_Third']/2 + dict1['Other']['Lower_Third']/2)
   
    data1 = [[path, str('Conflict with Upper and Middle'), float(Middle_Lower_blue), float(Middle_Lower_green), float(Middle_Lower_Brown), float(Middle_Lower_other) ]]
    conflict_result = pd.DataFrame(data1, columns=['File_name', 'Conflict_Type', 'Blue', 'Green', 'Brown','Other']).T
    print (conflict_result)
   
# 5 Upper Middle Bottom -Conflict

elif all_conflict is False:
   
    conflict_Blue  = (1*(dict1['Blue']['Upper_Third'] + dict1['Blue']['Lower_Third'])+ 2*(dict1['Blue']['Middle_Third']))/4
    conflict_Green = (1*(dict1['Green']['Upper_Third'] + dict1['Green']['Lower_Third'])+ 2*(dict1['Green']['Middle_Third']))/4
    conflict_Brown = (1*(dict1['Brown']['Upper_Third'] + dict1['Brown']['Lower_Third'])+ 2*(dict1['Brown']['Middle_Third']))/4
    conflict_other = (1*(dict1['Other']['Upper_Third'] + dict1['Other']['Lower_Third'])+ 2*(dict1['Other']['Middle_Third']) )/4

    data1 = [[path, str('All_conflict'), float(conflict_Blue), float(conflict_Green), float(conflict_Brown), float(conflict_other) ]]
    conflict_result = pd.DataFrame(data1, columns=['File_name', 'Conflict_Type', 'Blue', 'Green', 'Brown','Other']).T

    print (conflict_result)

# 4 Top and Bottom Agree but conflict in middle

elif Upper_Lower is False and Upper_Lower or Upper_Middle is True:
    
    conflict_Blue  = (1*(dict1['Blue']['Upper_Third'] + dict1['Blue']['Lower_Third'])+ 2*(dict1['Blue']['Middle_Third']))/4
    conflict_Green = (1*(dict1['Green']['Upper_Third'] + dict1['Green']['Lower_Third'])+ 2*(dict1['Green']['Middle_Third']))/4
    conflict_Brown = (1*(dict1['Brown']['Upper_Third'] + dict1['Brown']['Lower_Third'])+ 2*(dict1['Brown']['Middle_Third']))/4
    conflict_other = (1*(dict1['Other']['Upper_Third'] + dict1['Other']['Lower_Third'])+ 2*(dict1['Other']['Middle_Third']) )/4

   
    data1 = [[path, str('Conflict in Middle'), float(conflict_Blue), float(conflict_Green), float(conflict_Brown), float(conflict_other) ]]
    conflict_result = pd.DataFrame(data1, columns=['File_name', 'Conflict_Type', 'Blue', 'Green', 'Brown','Other']).T

    print (conflict_result)

                                                               0
File_name      /home/roopesh/Desktop/New_Corpus/Brown/front (...
Conflict_Type                                       All_conflict
Blue                                                         0.0
Green                                                        0.0
Brown                                                       62.0
Other                                                       37.5


In [97]:
all_conflict

False

In [98]:
Upper_Lower

False

In [109]:
data1 = [[path, str('Conflict with Upper and Middle'), float(conflict_Blue), float(conflict_Green), float(conflict_Brown), float(conflict_Other) ]]
conflict_result = pd.DataFrame(data1, columns=['File_name', 'Conflict_Type', 'Blue', 'Green', 'Brown','Other']).T


In [110]:
conflict_result

,0
File_name,/home/roopesh/Desktop/New_Corpus/Brown/front (...
Conflict_Type,All_conflict
Blue,24.75
Green,0.0
Brown,45.5
Other,28.75


In [133]:
import cv2
import cv_exp.basic
import cv_exp.basic as cv
import cv_exp.pupil_detection as pupil_detection
import cv_exp.draw as draw
import cv_exp.log as log
import cv_exp.take as take
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os
from PIL import Image
import pandas as pd
from scipy.cluster.vq import kmeans, vq
import matplotlib.pyplot as plt
import plotly.express as px


model_face_detection = '/home/roopesh/Desktop/MediaPipe_new_cv_exp/cv-exp-framework-master/data/models/face_detection_front.tflite'
model_face_landmarks ='/home/roopesh/Desktop/MediaPipe_new_cv_exp/cv-exp-framework-master/data/models/face_landmark.tflite'
model_iris_landmarks = '/home/roopesh/Desktop/MediaPipe_new_cv_exp/cv-exp-framework-master/data/models/iris_landmark.tflite'

iris_detector = pupil_detection.IrisDetectorMP(model_face_detection_path=model_face_detection,
                                                   model_face_landmarks_path=model_face_landmarks,
                                                   model_iris_landmarks_path=model_iris_landmarks)

# def Mediapipe_Iris_Color(path):
#     '''

#     This function takes the image in path and Using Mediapipe calculates the Annular Mask of the images,
#     then calculates the HSV color of Annular mask.
#     and returns the dominant Iris colour percentage in a data frame.
#     '''



def rgb_to_hsv(r, g, b):
    '''
    Convert RGB to HSV
    returns h,s,v
    '''

    r, g, b = r/255.0, g/255.0, b/255.0
    mx = max(r, g, b)
    mn = min(r, g, b)
    df = mx-mn
    if mx == mn:
        h = 0
    elif mx == r:
        h = (60 * ((g-b)/df) + 360) % 360
    elif mx == g:
        h = (60 * ((b-r)/df) + 120) % 360
    elif mx == b:
        h = (60 * ((r-g)/df) + 240) % 360
    if mx == 0:
        s = 0
    else:
        s = (df/mx)*100
    v = mx*100
    return h, s, v

# def Mediapipe_Iris_Color(path):
#     '''

#     This function takes the image in path and Using Mediapipe calculates the Annular Mask of the images,
#     then calculates the HSV color of Annular mask.
#     and returns the dominant Iris colour percentage in a data frame.
#     '''



path='/home/roopesh/Desktop/New_Corpus/Brown/front (3).jpg'

image = cv2.imread(path)
image_bgr_rgb = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # converting from BGR to RGB
out_ = image_bgr_rgb.copy()
out_pupile_ = image_bgr_rgb.copy()
result = iris_detector.detect(image_bgr_rgb)

if not result is None:

    detected_pupils = result.detected_pupils,
    face_rect_detection = result.face_rect_detection_from_landmarks,
    face_rect_landmarks = result.face_rect_from_landmarks,
    face_landmarks = result.face_landmarks,
    face_rect_detection_from = result.face_rect_from_detection,
    face_landmarks_from_detection = result.face_landmarks_from_detection,
    left_iris_landmarks = result.left_iris_landmarks,
    left_eyelid_landmarks = result.left_eyelid_landmarks,
    right_iris_landmarks = result.right_iris_landmarks,
    right_eyelid_landmarks = result.right_eyelid_landmarks,
    left_iris_segmentation = result.left_iris_segmentation,
    right_iris_segmentation = result.right_iris_segmentation,
    eyes_landmarks = result.eyes_landmarks

    # drawing (Optional)

    out_pupile = draw.draw_pupils(out_pupile_, result.detected_pupils.left, result.detected_pupils.right, color_center = (255, 255, 255), color_contour = (255, 255, 255))
    out_pupile = draw.draw_rect(out_pupile_, result.face_rect_detection_from_landmarks, thickness=6, color=(255, 0, 0))
    # out_pupile = draw.draw_list_pointd(out_pupile_, result.left_iris_landmarks,radius=6,color = (255, 255, 255))
    # out_pupile= draw.draw_contour(out_pupile_, result.left_iris_segmentation,  thickness=1,color=(0, 0, 255))
    # out_pupile = draw.draw_list_pointd(out_pupile_, result.left_eyelid_landmarks,radius=6,color = (255, 0, 0))

    # Extraction of specific Points From Left Iris Landmarks (0,1) : Left Iris from Coder's Point of view

    left_iris_tuple_of_tuple_Tolist = ([PointD_elements for PointD in left_iris_landmarks for PointD_elements in PointD])
    left_iris_points = [(pointD.x, pointD.y) for pointD in left_iris_tuple_of_tuple_Tolist]

    center_point_LeftIris=(np.round(left_iris_points[0]))
    for left_center_x,left_center_y in [center_point_LeftIris]:
         lx, ly = left_center_x,left_center_y

    left_point_LeftIris=(np.round(left_iris_points[1]))
    for leftIris_left_x,LeftIris_left_y in [left_point_LeftIris]:
        LIL_x, LIL_y = leftIris_left_x, LeftIris_left_y

    right_point_LeftIris=(np.round(left_iris_points[3]))
    for rightIris_x,rightIris_y in [right_point_LeftIris]:
        RIL_x, RIL_y = rightIris_x, rightIris_y

    Top_point_LeftIris=(np.round(left_iris_points[2]))
    for TopIris_x,TopIris_y in [Top_point_LeftIris]:
        TI_x, TI_y = TopIris_x, TopIris_y

    # Bottom_point_LeftIris=(np.round(left_iris_points[4]))
    # for BottomIris_x,BottomIris_y in [Bottom_point_LeftIris]:
    #     BI_x, BI_y = BottomIris_x, BottomIris_y


    # Distance Formula to calculate the distance between center to left_Iris_ points to get radius of iris
    distance_left_iris_center = np.sqrt((lx - LIL_x) ** 2 + (ly - LIL_y) ** 2)

    # First Circular mask for entire iris

    mask = np.zeros(out_pupile_.shape, dtype=np.uint8)      # Result from Mediapipe added here.
    center = (int(lx), int(ly))
    radius = (int((distance_left_iris_center) * 0.80))
    left_mask = cv2.circle(mask, center, radius,[255,255,255], -1 )
    result = cv2.bitwise_and(out_pupile_, left_mask)
    result[mask == 0] = 255                                 # adding background Color- white

    # cropping iris area from the first circular masked image

    x = int((RIL_x)+3)               # x-axis --Landmarks points from top- right point from left iris, +3 for adjusting the size
    y = int((TI_y)+3)                # y-axis --Landmarks points from top_ left of left iris, +3 for adjusting the size
    h = int(radius*2.35)-3               # height --Radius of iris times 2.35 for adjustment,-3 for adjustment
    w = int(radius*2.35)             # width  --Radius of iris times 2.35 for adjustment, -1 for adjustment
    crop = result[y:y+h, x:x+w]

    # second Circular mask to take inner pupil area alone

    mask1 = np.zeros(out_pupile_.shape, dtype=np.uint8)         # Result from Mediapipe added here for second mask.
    center1 = (int(lx), int(ly))
    radius1 = (int((distance_left_iris_center)* 0.50))          # to scrap pupil area and other reflection area
    left_mask1 = cv2.circle(mask1, center1, radius1, (255, 255, 255), -1 )
    result1 = cv2.bitwise_and(out_pupile_, left_mask1-255)      # subtracting white region over masked area
    result1[mask1 == 0] = 255                                   # adding background Color- white

    # Cropping the inner pupil area from the second circular masked image (size should be same as the first masked and cropped result )

    x = int(RIL_x)+3         # x-axis --Landmarks points from top- right point from left iris, +3 for adjusting the size
    y = int(TI_y)+3          # y-axis --Landmarks points from top_ left of left iris, +3 for adjusting the size
    h = int(radius*2.35)-3   # height --Radius of iris times 2.35,-3 for adjustment
    w = int(radius*2.35)     # width  --Radius of iris times 2.35, -1 for adjustment
    crop1 = result1[y:y+h, x:x+w]

    # Bitwise and operation to omit the inner pupil alone and take the rest of the iris area

    adding_both_cropped_image = cv2.bitwise_and(crop, crop1)

    # Removing NOISE from the Adding_both_cropped_image (<=10,  >=240) in all channel

    final_result = np.array(adding_both_cropped_image)
    mask = np.all(final_result <= [10, 10, 10], axis=2)
    final_result[mask] = [0, 0, 0]
    mask = np.all(final_result >= [240, 240, 240], axis=2)
    final_result[mask] = [0, 0, 0]
    annular_result = final_result                   # the final result of the iris area from mediapipe

    # Additional Cropping the iris masked from the Annular_result to make 3 sub cropped images

    x, y = final_result.shape[:2]
    half = x//2
    top_third_= x//3
    Upper_Third = final_result[:top_third_]               # Upper third of the image
    top_3 = final_result[top_third_:half]
    bottom = final_result[half:]
    x1, y1 = bottom.shape[:2]
    bottom_third = x1//3
    bo_3 = bottom[:bottom_third]
    half1 = x1//3
    Lower_Third = bottom[half1:]                          # Lower third of the image
    Middle_Third = np.vstack((top_3, bo_3))               # middle third of the image

    dict01 = dict()
    dict01['Upper_Third'] = Upper_Third
    dict01['Middle_Third'] = Middle_Third
    dict01['Lower_Third'] = Lower_Third

    # Printing the cropped iris area from the Annular_result (optional)

    annular_result1 = cv2.cvtColor(annular_result, cv2.COLOR_RGB2BGR)
    Upper_Third1 = cv2.cvtColor(Upper_Third, cv2.COLOR_RGB2BGR)
    Middle_Third1 = cv2.cvtColor(Middle_Third, cv2.COLOR_RGB2BGR)
    Lower_Third1 = cv2.cvtColor(Lower_Third, cv2.COLOR_RGB2BGR)

    out_pupile_1 = cv2.cvtColor(out_pupile, cv2.COLOR_RGB2BGR)
    out_pupile_1 = cv2.putText(out_pupile_1, "Left Iris", (int(TI_x), int(TI_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

#     cv2.imwrite(f'/home/roopesh/Desktop/new_cvexp_results/V2.0/folder/annular_Iris_/{file}',annular_result1)
#     cv2.imwrite(f'/home/roopesh/Desktop/new_cvexp_results/V2.0/folder/Iris_original_/{file}',out_pupile_1)
#     cv2.imwrite(f'/home/roopesh/Desktop/new_cvexp_results/V2.0/folder/Upper_Third/{file}',Upper_Third1)
#     cv2.imwrite(f'/home/roopesh/Desktop/new_cvexp_results/V2.0/folder/Middle_Third/{file}',Middle_Third1)
#     cv2.imwrite(f'/home/roopesh/Desktop/new_cvexp_results/V2.0/folder/Lower_Third/{file}',Lower_Third1)


    #   Reading Image in PIL format for dominant color detection.

    dominantColour = []

    for keys, values in dict01.items():
        pil_image = Image.fromarray(values)
        imarray = np.array(pil_image.getdata())
        df = pd.DataFrame(imarray, columns=['RED', 'GREEN', 'BLUE'])
        df = pd.DataFrame(df[(df['RED'] >= 50) | (df['GREEN'] >= 50) | (df['BLUE'] >= 50)])  # For value greater than 50

         # K-Means Clustering using Scipy

        cluster_size = 3
        cluster_centers, _ = kmeans(df.values.astype(float), cluster_size)
        index, _ = vq(df.values.astype(float), cluster_centers)
        count = np.bincount(index)
        sum = np.sum(count)
        i, Blue, Green, Brown, Other = 0, 0, 0, 0, 0
        for row in cluster_centers:
            r = row[0]
            g = row[1]
            b = row[2]
            h, s, v = rgb_to_hsv(r, g, b)   # calling the function to convert RGB to HSV
            percent = count[i]*100/sum
            if h >= 170 and h < 270:
                Blue += percent
            elif h >= 60 and h < 170:
                Green += percent
            elif h >= 300 or h < 40:
                Brown += percent
            else:
                Other += percent
            i+=1

        # Results of Dominant Color Detection in dataframe

        data = [[path, str(keys), int(Blue), int(Green), int(Brown), int(Other) ]]
        dominantColour_ = pd.DataFrame(data, columns=['File_name', 'Image_Position', 'Blue', 'Green', 'Brown','Other']) 
        dominantColour.append(dominantColour_)
        
     
   return conflict_result

#         print(dominantColour[0])

    else: print("No face detected :", file)

     # **************************  END of Dominant Colour Detection **************************


    
def check_5Percent_range(x,y):
    flg=False
    if x < y:
        if x+(x*0.05) >= y:
            flg=True
    elif x > y:
        if x - (x*0.05) <= y:
            flg=True
    elif x==y:
        flg=True
    return flg
    
    

if __name__ == '__main__':

# ----------Load Image Directory -------------
# images_files_dir= '/home/roopesh/Desktop/new_cvexp_results/V2.0/New_Green_Eyes/Green_Iris_Training_Image/' # Directory of the images

# images_files_dir= '/home/roopesh/Desktop/new_cvexp_results/V2.0/New_Brown_Eyes/Brown_Iris_Training_Image_V4.0/'
# images_files_dir= '/home/roopesh/Desktop/new_cvexp_results/V2.0/New_Blue_Eyes/Blue_Iris_Training_Image (copy)/'
# images_files_dir= '/home/roopesh/Desktop/new_cvexp_results/V2.0/New_Green_Eyes/Green_Iris_Training_Image_V4.0/'

# images_files_dir= '/home/roopesh/Desktop/New_Corpus_v3/Brown_V4.0/'
# images_files_dir= '/home/roopesh/Desktop/New_Corpus_v3/Blue/'
    images_files_dir= '/home/roopesh/Desktop/New_Corpus_v3/Green/'

    files = os.listdir(images_files_dir)
    # sorted_file=sorted(files, key=lambda x : int(x[5:-4]))
    for idx, file in enumerate(files):
        image_path = images_files_dir + file
        # print('debug', file)
        mediapipe_iris_color_chart = Mediapipe_Iris_Color(image_path)           # Calling the Mediapipe_Iris_Color(image_path)
        print(mediapipe_iris_color_chart)





{'File_name': {0: '/home/roopesh/Desktop/New_Corpus/Brown/front (3).jpg'}, 'Image_Position': {0: 'Upper_Third'}, 'Blue': {0: 100}, 'Green': {0: 0}, 'Brown': {0: 0}, 'Other': {0: 0}}


KeyError: 'Upper_Third'

In [122]:
dict1

{'File_name': {'Upper_Third': '/home/roopesh/Desktop/New_Corpus/Brown/front (3).jpg',
  'Middle_Third': '/home/roopesh/Desktop/New_Corpus/Brown/front (3).jpg',
  'Lower_Third': '/home/roopesh/Desktop/New_Corpus/Brown/front (3).jpg'},
 'Blue': {'Upper_Third': 99, 'Middle_Third': 0, 'Lower_Third': 0},
 'Green': {'Upper_Third': 0, 'Middle_Third': 0, 'Lower_Third': 0},
 'Brown': {'Upper_Third': 0, 'Middle_Third': 70, 'Lower_Third': 42},
 'Other': {'Upper_Third': 0, 'Middle_Third': 29, 'Lower_Third': 57}}